# Aggregation pipeline
This Notebook is about getting to know Mongo's accident analysis pipeline. It's presented as a series of exercises that generally replicate queries you've already perfromed on the accidents dataset.

In [ ]:
# Import the required libraries

import pymongo
import datetime
import collections

import pandas as pd
import scipy.stats

In [ ]:
# Open a connection to the Mongo server, open the accidents database and name the collections of accidents and labels
client = pymongo.MongoClient('mongodb://localhost:27351/')

db = client.accidents
accidents = db.accidents
labels = db.labels

In [ ]:
# Load the expanded names of keys and human-readable codes into memory

expanded_name = collections.defaultdict(str)
for e in labels.find({'expanded': {"$exists": True}}):
    expanded_name[e['label']] = e['expanded']
    
label_of = collections.defaultdict(str)
for l in labels.find({'codes': {"$exists": True}}):
    for c in l['codes']:
        try:
            label_of[l['label'], int(c)] = l['codes'][c]
        except ValueError: 
            label_of[l['label'], c] = l['codes'][c]

## An example
First, an example of using the aggregation pipeline to get you started. 

This pipeline finds all the accidents at 30mph or above, groups them by speed limit, and finds the number of accidents at each speed limit.

* `$match` is first and selects the relevant accidents.
* `$group` is next and groups the accidents. The `_id` is the value being grouped on. `{'$sum': 1}` is the value assigned to each member of the group, and how those values are combined. This is the idiom for counting members in a group.

In [ ]:
# Pull out all the accidents at 30mph or above, group by speed, and show totals at each speed.
pipeline = [{'$match': {'Speed_limit': {'$gte': 30}}},
            {'$group': {'_id': '$Speed_limit',
                        'num_accidents': {'$sum': 1}}}]
results = list(accidents.aggregate(pipeline))
results

We can now put the results in a _pandas_ DataFrame.

In [ ]:
results_df = pd.DataFrame(results)
results_df.set_index('_id', inplace=True)
results_df.index.name = 'speed_limit'
results_df

### Activity note
With all the activities below, build up the pipeline in stages. The `$limit` operator is your friend here: it will allow you to see what the pipeline produces without being overwhelmed by thousands of items.

### Activity 1
Find all the accidents at 30mph or above, group them by speed limit and accident severity, and find the number of accidents at each speed limit/severity combination.

Hint: If you give multiple keys for a single `$group` operation, it will return one group for each combination of those keys.

The solution is in the [`15.2solutions`](15.2solutions.ipynb) Notebook.

In [ ]:
# Insert your solution here.

### Activity 2
Given the number of accidents at each speed limit/severity combination, create a *pandas* DataFrame to store the results. The DataFrame should have columns of severity (as 'Fatal', 'Serious', 'Slight', in that order) and an index of speed limits (in order).

Hint: Convert the result of Activity 1 into a list of dicts, import it into a DataFrame, then use `set_index` and `unstack` to reshape the data.

The solution is in the [`15.2solutions`](15.2solutions.ipynb) Notebook.

In [ ]:
# Insert your solution here.

### Activity 3
Group the accidents by number of vehicles and number of casualties. Find the number of accidents at each vehicle/casualty combination. Store the results in a DataFrame with the number of vehicles as the columns and number of casualties as the index. 

The solution is in the [`15.2solutions`](15.2solutions.ipynb) Notebook.

In [ ]:
# Insert your solution here.

### Activity 4
Group the accidents by severity and junction type (`Junction_Detail`). Find the number of accidents at each junction/severity combination. Store the results in a DataFrame with the severities as the columns and junction types as the index. Columns and index should contain the text labels (e.g. Fatal, Serious; Roundabout, Crossroads), not the codes.

The solution is in the [`15.2solutions`](15.2solutions.ipynb) Notebook.

In [ ]:
# Insert your solution here.

### Activity 5
Find the average number of vehicles and casualties for accidents at each speed limit. Replace the `_id` of each speed limit group with the plain `Speed_limit`.

Store the results in a DataFrame with the averages as the columns and speed limits as the index. 

Hint: Use `$group` to find the total vehicles and casualties, then use `$project` to find the averages and rename the ID.

The solution is in the [`15.2solutions`](15.2solutions.ipynb) Notebook.

In [ ]:
# Insert your solution here.

### Activity 6
Find number of casualties for each casualty severity / casualty age band combination. 

Store the results in a DataFrame with the severities as the columns and age bands as the index. The columns and index should contain the text labels (e.g. 21-25, 46-55; Fatal, Slight), not the codes.

Hint: Use `$unwind` to examine each `casualty` sub-document in turn.

The solution is in the [`15.2solutions`](15.2solutions.ipynb) Notebook.

In [ ]:
# Insert your solution here.

### Activity 7
We can use the aggregation pipeline for complex queries. In this exercise, you'll recreate the summary data for finding Spearman's *r* for passenger and driver age bands. 

Find the number of casualties for each driver age band / passenger age band combination. 

Be careful that you only count casualty/driver combinations where the casualty and driver are in the same vehicle. Make sure you exclude combinations where either the driver age band or passenger age band is unknown.

Store the results in a DataFrame with the driver age band as the columns and passenger age bands as the index. The columns and index should contain the text labels (e.g. 21-25, 46-55), not the codes.

Expand the summary results into a list of individual driver/passenger age bands and find Spearman's *r* for these data.

Hint: You can't directly `$match` two values in the same document, so use `$project`'s `$eq` to create a field that flags if a driver/casualty combination comes from one vehicle then `$match` on that field.

The solution is in the [`15.2solutions`](15.2solutions.ipynb) Notebook.

In [ ]:
# Insert your solution here.

## Summary
Aggregation pipelines allow complex data-processing tasks to be built up from sequences of simple operations. It continues the trend of moving the data processing to be closer to the data. We started by bringing the data into the client machine for processing (in the form of a spreadsheet or DataFrame). Simple database queries allowed us to bring just the data of interest to the client, but we still had to do the processing and summarisation here. Aggregation pipelines allow us to move much more of the processing to the database, meaning we can end up with just the summary data alone. As datasets get larger, this becomes more important.

## What next?
If you are working through this Notebook as part of an inline exercise, return to the module materials now.

If you are working through this set of Notebooks as a whole, move on to `15.3 Plotting small accidents and roads`.